In [1]:
# basic dependencies

import numpy as np
from numpy import loadtxt
from numpy import savetxt

import pandas as pd
import math
import time

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

###########

# torch dependencies
import torch

tkwargs = {"dtype": torch.double, # set as double to minimize zero error for cholesky decomposition error
           "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu")} # set tensors to GPU, if multiple GPUs please set cuda:x properly

torch.set_printoptions(precision=3)

###########

# botorch dependencies
import botorch

# data related
from botorch.utils.sampling import draw_sobol_samples
from botorch.utils.transforms import unnormalize, normalize

# surrogate model specific
from botorch.models.gp_regression import SingleTaskGP
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch import fit_gpytorch_model

# qNEHVI specific
from botorch.acquisition.multi_objective.objective import IdentityMCMultiOutputObjective
from botorch.acquisition.multi_objective.monte_carlo import qNoisyExpectedHypervolumeImprovement

# utilities
from botorch.optim.optimize import optimize_acqf
from botorch.sampling.samplers import SobolQMCNormalSampler
from botorch.utils.multi_objective.pareto import is_non_dominated
from botorch.utils.multi_objective.hypervolume import Hypervolume
from typing import Optional
from torch import Tensor
from botorch.exceptions import BadInitialCandidatesWarning

import warnings

warnings.filterwarnings('ignore', category=BadInitialCandidatesWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

###########

# pymoo dependencies
import pymoo

from pymoo.problems import get_problem
from pymoo.core.problem import ElementwiseProblem

from pymoo.algorithms.moo.unsga3 import UNSGA3
from pymoo.util.ref_dirs import get_reference_directions
from pymoo.optimize import minimize

from pymoo.core.problem import Problem as PymooProblem
from pymoo.core.termination import NoTermination

###########

# jmetalpy dependencies
from jmetal.core.problem import FloatProblem
from jmetal.core.solution import FloatSolution
from jmetal.util.termination_criterion import StoppingByEvaluations, TerminationCriterion
from jmetal.util.aggregative_function import Tschebycheff
from jmetal.operator import PolynomialMutation, DifferentialEvolutionCrossover
from jmetal.algorithm.multiobjective.moead import Permutation
from jmetal.algorithm.multiobjective import MOEADIEpsilon


###########

from scipy.stats import qmc
from scipy.stats import gaussian_kde # for density plot

###########

# plotting dependencies
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

# this is for the colorbar, you can change the cmap if you prefer other colour schemes
from matplotlib.cm import ScalarMappable
cm = plt.cm.get_cmap('viridis')

# function to return the std dev across runs
def ci(y, N_TRIALS):
    return 1.96 * y.std(axis=0) / np.sqrt(N_TRIALS)

SMALL_SIZE = 14
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [2]:
def optimize_qnehvi(problem, ref_point, initial_x, # must haves
                    N_BATCH, BATCH_SIZE, 
                    random_state=torch.randint(1000000, (1,)).item(), noise=0, verbose=False): # change noise here!
    
    print("Optimizing with Pure qNEHVI")

    t0 = time.time()

    # some initializing 
    torch.manual_seed(random_state) # gives a consistent seed based on the trial number
    hv=Hypervolume(ref_point=-ref_point) # sets the hv based on problem, flip since BoTorch takes maximisation
    hvs = [] # create a blank array to append the scores at each batch/iteration for that run
    
    ##########
    # generate initial training data for that run
    train_x = initial_x
    train_obj, train_con = problem.evaluate(train_x)

    # add noise, by default noise=0, so train_noisy = train, noise factor determines amt of std dev to add
    train_obj_noisy = train_obj + noise*torch.randn_like(train_obj)
    train_con_noisy = train_con + noise*torch.randn_like(train_con)
    
    ##########
    
    # normalize inputs to [0,1] first before feeding into model
    standard_bounds = torch.zeros(2, problem.n_var, **tkwargs)
    standard_bounds[1] = 1
    train_x_gp = normalize(train_x, problem.bounds)
    
    # form the output train_y data by concentenating ground truth train_obj with its feasibility value on the rightmost
    # this is necessary since the surrogate GpyTorch model needs to model BOTH obj and con for predicted candidates
    train_y = torch.cat([train_obj_noisy, train_con_noisy], dim=-1) # model takes noisy observations

    # define and train surrogate models for objective and constraint
    models = []
    for i in range(train_y.shape[-1]):
        models.append(SingleTaskGP(train_x_gp, train_y[..., i : i + 1], outcome_transform=Standardize(m=1)))
    model = ModelListGP(*models)
    mll = SumMarginalLogLikelihood(model.likelihood, model)
        
    ##########    
    
    def create_idxr(i):
        def idxr(Z):
            return Z[..., i]

        return idxr

    def create_idxrs():
        return [create_idxr(i=i) for i in range(problem.n_obj, problem.n_obj+problem.n_constr)]
    
    # original location for an extra HV check wrt to initial samples
    
    ########## ########## ########## start of iteration loop


    # training loop for N_BATCH iterations
    for iteration in range(1, N_BATCH + 1):    

        t3 = time.time()
                
        # fit the surrogate model
        fit_gpytorch_model(mll)    
                
        ##########
            
        # define the acqusition function for EIC if feas_weighting is false
        acq_func = qNoisyExpectedHypervolumeImprovement(
            model=model,
            ref_point=-ref_point, # for computing HV, must flip for BoTorch
            X_baseline=train_x_gp, # feed total list of train_x for this current iteration
            sampler=SobolQMCNormalSampler(num_samples=128),  # determines how candidates are randomly proposed before selection
            objective=IdentityMCMultiOutputObjective(outcomes=np.arange(problem.n_obj).tolist()), # optimize first n_obj col 
            constraints=create_idxrs(), # constraint on last n_constr col
            prune_baseline=True, cache_pending=True)  # options for improving qNEHVI, keep these on
        
        ##########
        
        # propose candidates given defined qNEHVI acq func given model and latest observed training data
        new_x, _ = optimize_acqf(
                        acq_function=acq_func,
                        bounds=standard_bounds, # since train_x was normalized
                        q=BATCH_SIZE, # no of candidates to propose in parallel
                        num_restarts=2, # no of restarts of raw_samples
                        raw_samples=256,  # pool of samples to choose the starting points from
                        options={"batch_limit": 5, "maxiter": 200}, # default arguments, not too sure about this yet
                        )

        # unormalize our training inputs back to original problem bounds
        new_x =  unnormalize(new_x.detach(), bounds=problem.bounds)

        # feed new proposed observations into objective func to get its new ground truth
        new_obj, new_con = problem.evaluate(new_x)

        # add noise, by default noise=0, so train_noisy = train, noise factor determines amt of std dev to add
        new_obj_noisy = new_obj + noise*torch.randn_like(new_obj)
        new_con_noisy = new_con + noise*torch.randn_like(new_con)

        # update training points by concatenating the new values into their respective tensors
        train_x = torch.cat([train_x, new_x])
        train_obj = torch.cat([train_obj, new_obj])
        train_con = torch.cat([train_con, new_con])
        train_obj_noisy = torch.cat([train_obj_noisy, new_obj_noisy])
        train_con_noisy = torch.cat([train_con_noisy, new_con_noisy])
        
        ##########
        
        # computing HV of current candidate list
        is_feas = (train_con <= 0).all(dim=-1) # check whether points fit ALL (.all) constraint criteria
        feas_train_obj = train_obj[is_feas] # take only points that fit the 1st check
        if feas_train_obj.shape[0] > 0:
            pareto_mask = is_non_dominated(feas_train_obj) # check for 2nd criteria: non-dominated, meaning new pareto optimal
            pareto_y = feas_train_obj[pareto_mask] # take only points that fit the 2nd check
            volume = hv.compute(pareto_y) # compute change in HV with new pareto optimal wrt to original ref point
        else:
            volume = 0.0
        
        hvs.append(volume)
        
        ##########

        # update the surrogate models for next iteration
        train_x_gp = normalize(train_x, problem.bounds) # dont forget to renormalize!
        train_y = torch.cat([train_obj_noisy, train_con_noisy], dim=-1) # model takes noisy observations

        models = []
        for i in range(train_y.shape[-1]):
            models.append(SingleTaskGP(train_x_gp, train_y[..., i : i + 1], outcome_transform=Standardize(m=1)))
        model = ModelListGP(*models)
        mll = SumMarginalLogLikelihood(model.likelihood, model)
        
        ##########
        
        t4 = time.time()
        if verbose:
            print(
                    f"Batch {iteration:>2} of {N_BATCH}: Hypervolume = "
                    f"{hvs[-1]:>4.2f}, "
                    f"time = {t4-t3:>4.2f}s.\n"
                    , end="")
            
        del new_x, new_obj, new_con, new_obj_noisy, new_con_noisy, train_y
        torch.cuda.empty_cache() # clear some memory here between each run/trial     
        
        ########## ########## ########## end of iteration loop

    t1 = time.time()
    print(f"Time taken in total: {t1-t0:>4.2f}s.")       
    
    # returns the HV score across iterations, total training set as an array
    return hvs, torch.hstack([train_x, train_obj, train_con]).cpu().numpy()

In [3]:
def optimize_hybrid_nsga(problem, ref_point, initial_x, # must haves
                         N_BATCH, BATCH_SIZE, 
                         random_state=torch.randint(1000000, (1,)).item(), noise=0, verbose=False): # change noise here!
    
    print("Optimizing with Hybrid qNEHVI + U-NSGA-III")

    t0 = time.time()

    torch.manual_seed(random_state) # gives a consistent seed based on the trial number
    hv=Hypervolume(ref_point=-ref_point) # sets the hv based on problem, flip since BoTorch takes maximisation
    hvs = [] # create a blank array to append the scores at each batch/iteration for that run
    
    pymooproblem = PymooProblem(n_var=problem.n_var, n_obj=problem.n_obj, n_constr=problem.n_constr, 
                  xl=np.zeros(problem.n_var), xu=np.ones(problem.n_var))

    ##########
    # generate initial training data for that run
    train_x = initial_x
    train_obj, train_con = problem.evaluate(train_x)

    # add noise, by default noise=0, so train_noisy = train, noise factor determines amt of std dev to add
    train_obj_noisy = train_obj + noise*torch.randn_like(train_obj)
    train_con_noisy = train_con + noise*torch.randn_like(train_con)

    ##########

    # normalize inputs to [0,1] first before feeding into model
    standard_bounds = torch.zeros(2, problem.n_var, **tkwargs)
    standard_bounds[1] = 1
    train_x_gp = normalize(train_x, problem.bounds)

    # form the output train_y data by concentenating ground truth train_obj with its feasibility value on the rightmost
    # this is necessary since the surrogate GpyTorch model needs to model BOTH obj and con for predicted candidates
    train_y = torch.cat([train_obj_noisy, train_con_noisy], dim=-1) # model takes noisy observations

    # define and train surrogate models for objective and constraint
    models = []
    for i in range(train_y.shape[-1]):
        models.append(SingleTaskGP(train_x_gp, train_y[..., i : i + 1], outcome_transform=Standardize(m=1)))
    model = ModelListGP(*models)
    mll = SumMarginalLogLikelihood(model.likelihood, model)
    
    ##########    
    
    def create_idxr(i):
        def idxr(Z):
            return Z[..., i]

        return idxr

    def create_idxrs():
        return [create_idxr(i=i) for i in range(problem.n_obj, problem.n_obj+problem.n_constr)]
    
    ########## ########## ########## start of iteration loop

    for iteration in range(1, N_BATCH + 1):   

        t3 = time.time()

        ##########

        fit_gpytorch_model(mll)  

        # define the acqusition function for EIC if feas_weighting is false
        acq_func = qNoisyExpectedHypervolumeImprovement(
            model=model,
            ref_point=-ref_point, # for computing HV, must flip for BoTorch
            X_baseline=train_x_gp, # feed total list of train_x for this current iteration
            sampler=SobolQMCNormalSampler(num_samples=128),  # determines how candidates are randomly proposed before selection
            objective=IdentityMCMultiOutputObjective(outcomes=np.arange(problem.n_obj).tolist()), # optimize first n_obj col 
            constraints=create_idxrs(), # constraint on last n_constr col
            prune_baseline=True, cache_pending=True)  # options for improving qNEHVI, keep these on

        # propose best candidates given QMC and qNEHVI
        qnehvi_x, _ = optimize_acqf(acq_function=acq_func,
                                    bounds=standard_bounds, # since train_x was normalized
                                    q=BATCH_SIZE, # no of candidates to propose in parallel, 12 is the max for a GTX1065
                                    num_restarts=1, # no of restarts if q candidates fail to show improvement
                                    raw_samples=256,  # pool of samples to choose the starting points from
                                    options={"batch_limit": 5, "maxiter": 200}, # default arguments, not too sure about this yet
                                 )

        ##########
       
        algorithm = UNSGA3(pop_size=256,
                           ref_dirs=get_reference_directions("energy", problem.n_obj, 256, seed=random_state),
                           sampling=train_x_gp.cpu().numpy(),
                          )

        algorithm.setup(pymooproblem, termination=NoTermination())

        # set the 1st population to the current evaluated population
        pop = algorithm.ask()
        pop.set("F", train_obj_noisy.cpu().numpy())
        pop.set("G", train_con_noisy.cpu().numpy())
        algorithm.tell(infills=pop)

        # propose children based on tournament selection -> crossover/mutation
        newpop = algorithm.ask()
        nsga3_x = torch.tensor(newpop.get("X"), **tkwargs)
        
        ##########

        candidates = torch.cat([qnehvi_x, nsga3_x])

        acq_value_list = []

        for i in range(0, candidates.shape[0]):
            with torch.no_grad():
                acq_value = acq_func(candidates[i].unsqueeze(dim=0))
                acq_value_list.append(acq_value.item())

        pred_hv_list = []
        model.eval();

        for i in range(0, candidates.shape[0]):
            with torch.no_grad():
                posterior = model.posterior(candidates[i].unsqueeze(0))
                pred_y = posterior.mean
                pred_hv = hv.compute(pred_y[:,:problem.n_obj])
                pred_hv_list.append(pred_hv)

        sorted_x = candidates.cpu().numpy()[np.lexsort((pred_hv_list, acq_value_list))]

        ##########
        
        new_x = torch.tensor(sorted_x[-BATCH_SIZE:], **tkwargs) # take best BATCH_SIZE samples
        new_x =  unnormalize(new_x.detach(), bounds=problem.bounds)
        new_obj, new_con = problem.evaluate(new_x)

        # add noise, by default noise=0, so train_noisy = train, noise factor determines amt of std dev to add
        new_obj_noisy = new_obj + noise*torch.randn_like(new_obj)
        new_con_noisy = new_con + noise*torch.randn_like(new_con)

        # update training points by concatenating the new values into their respective tensors
        train_x = torch.cat([train_x, new_x])
        train_obj = torch.cat([train_obj, new_obj])
        train_con = torch.cat([train_con, new_con])
        train_obj_noisy = torch.cat([train_obj_noisy, new_obj_noisy])
        train_con_noisy = torch.cat([train_con_noisy, new_con_noisy])

        # computing HV of current candidate list
        is_feas = (train_con <= 0).all(dim=-1) # check whether points fit ALL (.all) constraint criteria
        feas_train_obj = train_obj[is_feas] # take only points that fit the 1st check
        if feas_train_obj.shape[0] > 0:
            pareto_mask = is_non_dominated(feas_train_obj) # check for 2nd criteria: non-dominated, meaning new pareto optimal
            pareto_y = feas_train_obj[pareto_mask] # take only points that fit the 2nd check
            volume = hv.compute(pareto_y) # compute change in HV with new pareto optimal wrt to original ref point
        else:
            volume = 0.0

        hvs.append(volume)

        ##########

        # update the surrogate models for next iteration
        train_x_gp = normalize(train_x, problem.bounds) # dont forget to renormalize!
        train_y = torch.cat([train_obj_noisy, train_con_noisy], dim=-1) # model takes noisy observations

        models = []
        for i in range(train_y.shape[-1]):
            models.append(SingleTaskGP(train_x_gp, train_y[..., i : i + 1], outcome_transform=Standardize(m=1)))
        model = ModelListGP(*models)
        mll = SumMarginalLogLikelihood(model.likelihood, model)

        ##########

        t4 = time.time()
        if verbose:
            print(
                    f"Batch {iteration:>2} of {N_BATCH}: Hypervolume = "
                    f"{hvs[-1]:>4.2f}, "
                    f"time = {t4-t3:>4.2f}s.\n"
                    , end="")

        del new_x, new_obj, new_con, qnehvi_x, nsga3_x, new_obj_noisy, new_con_noisy, train_y
        torch.cuda.empty_cache() # clear some memory here between each run/trial    
        
    t1 = time.time()
    print(f"Time taken in total: {t1-t0:>4.2f}s.")   
    
    del models, model, mll
    torch.cuda.empty_cache() # clear some memory here between each run/trial  
    
    # returns the HV score across iterations, total training set as an array
    return hvs, torch.hstack([train_x, train_obj, train_con]).cpu().numpy()

In [4]:
def optimize_hybrid_moead(problem, ref_point, initial_x, # must haves
                         N_BATCH, BATCH_SIZE, 
                         random_state=torch.randint(1000000, (1,)).item(), noise=0, verbose=False): # change noise here!
    
    print("Optimizing with Hybrid qNEHVI + MOEAD")

    t0 = time.time()

    torch.manual_seed(random_state) # gives a consistent seed based on the trial number
    hv=Hypervolume(ref_point=-ref_point) # sets the hv based on problem, flip since BoTorch takes maximisation
    hvs = [] # create a blank array to append the scores at each batch/iteration for that run
    
    # define jmetal class
    class jmetalproblem(FloatProblem):

        def __init__(self):
            super(jmetalproblem, self).__init__()
            self.number_of_variables = problem.n_var
            self.number_of_objectives = problem.n_obj
            self.number_of_constraints = problem.n_constr

            obj_directions = []
            obj_labels = []

            for i in range(1,problem.n_obj+1):
                obj_directions.append(self.MINIMIZE)
                obj_labels.append(f'f{i}')

            self.obj_directions = obj_directions
            self.obj_labels = obj_labels

            self.lower_bound = [0.0] * problem.n_var
            self.upper_bound = [1.0] * problem.n_var

        def evaluate(self, solution: FloatSolution) -> FloatSolution:
            pass

        def get_name(self):
            return 'jmetalproblem'

    ##########
    # generate initial training data for that run
    train_x = initial_x
    train_obj, train_con = problem.evaluate(train_x)

    # add noise, by default noise=0, so train_noisy = train, noise factor determines amt of std dev to add
    train_obj_noisy = train_obj + noise*torch.randn_like(train_obj)
    train_con_noisy = train_con + noise*torch.randn_like(train_con)

    ##########

    # normalize inputs to [0,1] first before feeding into model
    standard_bounds = torch.zeros(2, problem.n_var, **tkwargs)
    standard_bounds[1] = 1
    train_x_gp = normalize(train_x, problem.bounds)

    # form the output train_y data by concentenating ground truth train_obj with its feasibility value on the rightmost
    # this is necessary since the surrogate GpyTorch model needs to model BOTH obj and con for predicted candidates
    train_y = torch.cat([train_obj_noisy, train_con_noisy], dim=-1) # model takes noisy observations

    # define and train surrogate models for objective and constraint
    models = []
    for i in range(train_y.shape[-1]):
        models.append(SingleTaskGP(train_x_gp, train_y[..., i : i + 1], outcome_transform=Standardize(m=1)))
    model = ModelListGP(*models)
    mll = SumMarginalLogLikelihood(model.likelihood, model)
    
    ##########    
    
    def create_idxr(i):
        def idxr(Z):
            return Z[..., i]

        return idxr

    def create_idxrs():
        return [create_idxr(i=i) for i in range(problem.n_obj, problem.n_obj+problem.n_constr)]
    
    ########## ########## ########## start of iteration loop

    for iteration in range(1, N_BATCH + 1):   

        t3 = time.time()

        ##########

        fit_gpytorch_model(mll)  

        # define the acqusition function for EIC if feas_weighting is false
        acq_func = qNoisyExpectedHypervolumeImprovement(
            model=model,
            ref_point=-ref_point, # for computing HV, must flip for BoTorch
            X_baseline=train_x_gp, # feed total list of train_x for this current iteration
            sampler=SobolQMCNormalSampler(num_samples=128),  # determines how candidates are randomly proposed before selection
            objective=IdentityMCMultiOutputObjective(outcomes=np.arange(problem.n_obj).tolist()), # optimize first n_obj col 
            constraints=create_idxrs(), # constraint on last n_constr col
            prune_baseline=True, cache_pending=True)  # options for improving qNEHVI, keep these on

        # propose best candidates given QMC and qNEHVI
        qnehvi_x, _ = optimize_acqf(acq_function=acq_func,
                                    bounds=standard_bounds, # since train_x was normalized
                                    q=BATCH_SIZE, # no of candidates to propose in parallel, 12 is the max for a GTX1065
                                    num_restarts=1, # no of restarts if q candidates fail to show improvement
                                    raw_samples=256,  # pool of samples to choose the starting points from
                                    options={"batch_limit": 5, "maxiter": 200}, # default arguments, not too sure about this yet
                                 )

        ##########
        
        max_pop = train_x_gp.shape[0]
        
        # redefine a new algo at each iteration
        MOEAD_algo = MOEADIEpsilon(problem=jmetalproblem(),
                          population_size=max_pop,
                          crossover=DifferentialEvolutionCrossover(CR=1.0, F=0.5, K=0.5),
                          mutation=PolynomialMutation(probability=1.0 / jmetalproblem().number_of_variables, distribution_index=20),
                          aggregative_function=Tschebycheff(dimension=jmetalproblem().number_of_objectives),
                          neighbor_size=int(max_pop/2),
                          neighbourhood_selection_probability=0.9,
                          max_number_of_replaced_solutions=2,
                          weight_files_path='weights',
                          #termination_criterion=StoppingByEvaluations(BATCH_SIZE),
                         )
        
        
        # initialize population
        initial_solutions = []

        for i in range(0, max_pop):
            solution1 = FloatSolution(lower_bound=[0.0] * problem.n_var,
                                      upper_bound=[1.0] * problem.n_var,
                                      number_of_objectives=problem.n_obj,
                                      number_of_constraints=problem.n_constr,
                                     )
            solution1.variables = train_x_gp[i].cpu().tolist()
            solution1.objectives = train_obj_noisy[i].cpu().tolist()
            solution1.constraints = train_con_noisy[i].cpu().tolist()
            
            initial_solutions.append(solution1)

        MOEAD_algo.solutions = initial_solutions

        # perform selection, crossover and mutation to form our offspring population
        MOEAD_algo.permutation = Permutation(max_pop)
        offspring_population = []

        for q in range(256):
            mating_population = MOEAD_algo.selection(MOEAD_algo.solutions)
            offspring = MOEAD_algo.reproduction(mating_population)
            offspring_population.append(offspring[0].variables)
            
        moead_x = torch.tensor(offspring_population, **tkwargs)
        
        ##########

        candidates = torch.cat([qnehvi_x, moead_x])

        acq_value_list = []

        for i in range(0, candidates.shape[0]):
            with torch.no_grad():
                acq_value = acq_func(candidates[i].unsqueeze(dim=0))
                acq_value_list.append(acq_value.item())

        pred_hv_list = []
        model.eval();

        for i in range(0, candidates.shape[0]):
            with torch.no_grad():
                posterior = model.posterior(candidates[i].unsqueeze(0))
                pred_y = posterior.mean
                pred_hv = hv.compute(pred_y[:,:problem.n_obj])
                pred_hv_list.append(pred_hv)

        sorted_x = candidates.cpu().numpy()[np.lexsort((pred_hv_list, acq_value_list))]

        ##########
        
        new_x = torch.tensor(sorted_x[-BATCH_SIZE:], **tkwargs) # take best BATCH_SIZE samples
        # unormalize our training inputs back to original problem bounds
        new_x =  unnormalize(new_x.detach(), bounds=problem.bounds)

        # feed new proposed observations into objective func to get its new ground truth
        new_obj, new_con = problem.evaluate(new_x)

        # add noise, by default noise=0, so train_noisy = train, noise factor determines amt of std dev to add
        new_obj_noisy = new_obj + noise*torch.randn_like(new_obj)
        new_con_noisy = new_con + noise*torch.randn_like(new_con)

        # update training points by concatenating the new values into their respective tensors
        train_x = torch.cat([train_x, new_x])
        train_obj = torch.cat([train_obj, new_obj])
        train_con = torch.cat([train_con, new_con])
        train_obj_noisy = torch.cat([train_obj_noisy, new_obj_noisy])
        train_con_noisy = torch.cat([train_con_noisy, new_con_noisy])

        # computing HV of current candidate list
        is_feas = (train_con <= 0).all(dim=-1) # check whether points fit ALL (.all) constraint criteria
        feas_train_obj = train_obj[is_feas] # take only points that fit the 1st check
        if feas_train_obj.shape[0] > 0:
            pareto_mask = is_non_dominated(feas_train_obj) # check for 2nd criteria: non-dominated, meaning new pareto optimal
            pareto_y = feas_train_obj[pareto_mask] # take only points that fit the 2nd check
            volume = hv.compute(pareto_y) # compute change in HV with new pareto optimal wrt to original ref point
        else:
            volume = 0.0

        hvs.append(volume)

        ##########

        # update the surrogate models for next iteration
        train_x_gp = normalize(train_x, problem.bounds) # dont forget to renormalize!
        train_y = torch.cat([train_obj_noisy, train_con_noisy], dim=-1) # model takes noisy observations

        models = []
        for i in range(train_y.shape[-1]):
            models.append(SingleTaskGP(train_x_gp, train_y[..., i : i + 1], outcome_transform=Standardize(m=1)))
        model = ModelListGP(*models)
        mll = SumMarginalLogLikelihood(model.likelihood, model)

        ##########

        t4 = time.time()
        if verbose:
            print(
                    f"Batch {iteration:>2} of {N_BATCH}: Hypervolume = "
                    f"{hvs[-1]:>4.2f}, "
                    f"time = {t4-t3:>4.2f}s.\n"
                    , end="")

        del new_x, new_obj, new_con, qnehvi_x, moead_x, new_obj_noisy, new_con_noisy, train_y
        torch.cuda.empty_cache() # clear some memory here between each run/trial    
        
    t1 = time.time()
    print(f"Time taken in total: {t1-t0:>4.2f}s.")   
    
    del models, model, mll
    torch.cuda.empty_cache() # clear some memory here between each run/trial  
    
    # returns the HV score across iterations, total training set as an array
    return hvs, torch.hstack([train_x, train_obj, train_con]).cpu().numpy()

In [5]:
'''dimensions = 8
    
from botorch.test_functions.multi_objective import MW7

MW7base = MW7(dim=dimensions, negate=True).to(**tkwargs)

class Problem_MW7(torch.nn.Module):
    n_var = dimensions
    n_obj = 2
    n_constr = 2
    
    ref_point = torch.tensor([1.2,1.2], **tkwargs)
    
    bounds = MW7base.bounds    
    
    def evaluate(X):       
        output = MW7base(X)
        slack = -MW7base.evaluate_slack(X)
       
        return output, slack

from botorch.test_functions.multi_objective import WeldedBeam

WeldedBeambase = WeldedBeam(negate=True).to(**tkwargs)

class Problem_WeldedBeam(torch.nn.Module):
    n_var = WeldedBeambase.dim
    n_obj = WeldedBeambase.num_objectives
    n_constr = WeldedBeambase.num_constraints
    
    ref_point = torch.tensor([40,0.015], **tkwargs)
    
    bounds = WeldedBeambase.bounds    
    
    def evaluate(X):       
        output = WeldedBeambase(X)
        slack = -WeldedBeambase.evaluate_slack(X)
       
        return output, slack
    
# this is equivalent to difficulty triplet no4 in pymoo's API
diff1 = 0.25
diff2 = 0.25
diff3 = 0.25

class Problem_DAS1(torch.nn.Module):
    n_var = dimensions
    n_obj = 2
    n_constr = 11
    bounds = torch.stack([torch.zeros(dimensions, **tkwargs),torch.ones(dimensions, **tkwargs)])
    
    # this will change depending on difficulty triplet
    # check the actual PF from pymoo to find
    ref_point = torch.tensor([1.6,1.6], **tkwargs)
    
    def evaluate(X):
        g = torch.zeros(X.shape[0], **tkwargs)
        for i in range(X.shape[1]):
            g += torch.pow(X[...,i] - torch.sin(0.5 * np.pi * X[...,0]), 2)

        f1 = X[...,0] + g
        f2 = 1.0 - torch.pow(X[...,0], 2) + g
        
        #################
        
        a = 20
        b = 2 * diff1 - 1.
        d = 0.5 if diff2 != 0 else 0
        if diff2 > 0:
            e = d - torch.log(torch.tensor(diff2, **tkwargs))
        else:
            e = 1e30
        r = 0.5 * diff3

        p_k = [0., 1.0, 0., 1.0, 2.0, 0., 1.0, 2.0, 3.0]
        q_k = [1.5, 0.5, 2.5, 1.5, 0.5, 3.5, 2.5, 1.5, 0.5]

        a_k2 = 0.3
        b_k2 = 1.2
        theta_k = torch.tensor(-0.25 * torch.pi, **tkwargs)

        c = torch.zeros((X.shape[0], 11), **tkwargs)

        c[...,0] = torch.sin(a * torch.pi * X[...,0]) - b

        if diff2 == 1:
            c[...,1] = 1e-4 - torch.abs(e - g, **tkwargs)
        else:
            c[...,1] = (e - g) * (g - d)

        for i in range(2, 11):
            c[...,i] = ((f1 - p_k[i-2]) * torch.cos(theta_k) - (f2 - q_k[i-2]) * torch.sin(theta_k)) ** 2 / a_k2 + ((f1 - p_k[i-2]) * torch.sin(theta_k) + (f2 - q_k[i-2]) * torch.cos(theta_k)) ** 2 / b_k2 - r


        ####################
        
        # to maximise a postive output, stay positive
        # to minimise a positive output, force negative
        # to maximise a negative output (away from zero), force positive
        # to minimise a negative output (towards zero), stay negative
        output = torch.stack([-f1, -f2], dim=-1)
        
        # constraint equations should take the form of c(n)<=0
        # should return <=0 values when constraints are active,
        # and >0 values when constraints are violated
        slack = -c    
        
        return output, slack
    
class MWBase(torch.nn.Module):   
    def g1(X, n_obj):
        d = dimensions
        n = d - n_obj

        z = torch.pow(X[..., n_obj - 1:], n)
        i = torch.arange(n_obj - 1, d, **tkwargs)

        exp = 1 - torch.exp(-10.0 * (z - 0.5 - i / (2 * d)) * (z - 0.5 - i / (2 * d)))
        distance = 1 + exp.sum(axis=1)
        return distance
    
    def g3(X, n_obj):
        contrib = 2.0 * torch.pow(
            X[..., n_obj - 1:] + (X[..., n_obj - 2:-1] - 0.5) * (X[..., n_obj - 2:-1] - 0.5) - 1.0, 2.0)
        
        distance = 1 + contrib.sum(axis=1)
        return distance
    
    def LA1(A, B, C, D, theta):
        return A * torch.pow(torch.sin(B * np.pi * torch.pow(theta, C)), D)

    def LA2(A, B, C, D, theta):
        return A * torch.pow(torch.sin(B * torch.pow(theta, C)), D)
    
class Problem_MW3(torch.nn.Module):
    # must define these!
    n_var = dimensions
    n_obj = 2
    n_constr = 2 # inequality constraints only!
    
    ref_point = torch.tensor([1.1,1.1], **tkwargs)
        
    # input bounds, don't forget to change according to n_var!  
    bounds = torch.vstack([torch.zeros(dimensions, **tkwargs),torch.ones(dimensions, **tkwargs)])
    
    def evaluate(X):
        
        # objectives are defined by f1, f2....
        # if you have complex equations, you can break it up and place them in ProblemBase for convenience, not compulsory
        # otherwise, you could place an oracle here that outputs the objective
        g = MWBase.g3(X, n_obj=2)
        f1 = X[:, 0]
        f2 = g * (1.0 - f1 / g)
        
        # inequality constraints should take the form of c(n)<=0
        # for equality constraints, either use the equality constr argument, or rewrite into inequality form (preferred)
        # if no constraints, write as c1 = 0 * X[..., 0], and set n_constr=1     
        c1 = f1 + f2 - 1.05 - MWBase.LA1(0.45, 0.75, 1.0, 6.0, np.sqrt(2.0) * f2 - np.sqrt(2.0) * f1)
        c2 = 0.85 - f1 - f2 + MWBase.LA1(0.3, 0.75, 1.0, 2.0, np.sqrt(2.0) * f2 - np.sqrt(2.0) * f1)
        
        # botorch assumes maximisation
        # to maximise a postive output, stay positive
        # to minimise a positive output, force negative
        # to maximise a negative output (away from zero), force positive
        # to minimise a negative output (towards zero), stay negative
        output = torch.stack([-f1, -f2], dim=-1)
        
        # for 1 constraint, take c1.unsqueeze(dim=-1)
        # for >1 constraint, take torch.stack([c1, c2....], dim=-1)
        slack = torch.stack([c1, c2], dim=-1)
        
        return output, slack
    
class Problem_MW5(torch.nn.Module):
    n_var = dimensions
    n_obj = 2
    n_constr = 3
    
    ref_point = torch.tensor([1.2,1.2], **tkwargs)

    bounds = torch.vstack([torch.zeros(dimensions, **tkwargs),torch.ones(dimensions, **tkwargs)])
    
    def evaluate(X):
        # to maximise a postive output, stay positive
        # to minimise a positive output, force negative
        # to maximise a negative output (away from zero), force positive
        # to minimise a negative output (towards zero), stay negative
        
        g1 = MWBase.g3(X, n_obj=2)
        f1 = g1 * X[:, 0]
        f2 = g1 * torch.sqrt(1.0 - torch.pow(f1 / g1, 2.0))
                
        # constraint equations should take the form of c(n)<=0
        # should return <=0 values when constraints are active,
        # and >0 values when constraints are violated
        # take care if constraints are too restrictive, will cause early flatlining if all samples are rejected
        atan = torch.arctan(f2 / f1)
        c1 =  (f1 ** 2) + (f2 ** 2) - torch.pow(1.7 - MWBase.LA2(0.2, 2.0, 1.0, 1.0, atan), 2.0) 

        t = 0.5 * np.pi - 2 * torch.abs(atan - 0.25 * np.pi)
        c2 = torch.pow(1 + MWBase.LA2(0.5, 6.0, 3.0, 1.0, t), 2.0) - (f1 ** 2) - (f2 ** 2)
        c3 = torch.pow(1 - MWBase.LA2(0.45, 6.0, 3.0, 1.0, t), 2.0) - (f1 ** 2) - (f2 ** 2)
        
        output = torch.stack([-f1, -f2], dim=-1)
        slack = torch.stack([c1, c2, c3], dim=-1)
        
        return output, slack

class Problem_MW12(torch.nn.Module):
    # must define these!
    n_var = dimensions
    n_obj = 2
    n_constr = 2 # inequality constraints only!
    
    ref_point = torch.tensor([1.5,1.5], **tkwargs)
        
    # input bounds, don't forget to change according to n_var!  
    bounds = torch.vstack([torch.zeros(dimensions, **tkwargs),torch.ones(dimensions, **tkwargs)])
    
    def evaluate(X):
        
        # objectives are defined by f1, f2....
        # if you have complex equations, you can break it up and place them in ProblemBase for convenience, not compulsory
        # otherwise, you could place an oracle here that outputs the objective
        g = MWBase.g1(X, n_obj=2)
        f1 = g * X[:, 0]
        f2 = g * (0.85 - 0.8 * (f1 / g) - 0.08 *torch.abs(torch.sin(3.2 * torch.pi * (f1 / g))))
        
        # inequality constraints should take the form of c(n)<=0
        # for equality constraints, either use the equality constr argument, or rewrite into inequality form (preferred)
        # if no constraints, write as c1 = 0 * X[..., 0], and set n_constr=1     
        c1 = -1.0 * (1 - 0.625 * f1 - f2 + 0.08 * torch.sin(2 * torch.pi * (f2 - f1 / 1.6))) * (1.4 - 0.875 * f1 - f2 + 0.08 * torch.sin(2 * torch.pi * (f2 / 1.4 - f1 / 1.6)))
        c2 = (1 - 0.8 * f1 - f2 + 0.08 * torch.sin(2 * torch.pi * (f2 - f1 / 1.5))) * (1.8 - 1.125 * f1 - f2 + 0.08 * torch.sin(2 * torch.pi * (f2 / 1.8 - f1 / 1.6)))
        
        # botorch assumes maximisation
        # to maximise a postive output, stay positive
        # to minimise a positive output, force negative
        # to maximise a negative output (away from zero), force positive
        # to minimise a negative output (towards zero), stay negative
        output = torch.stack([-f1, -f2], dim=-1)
        
        # for 1 constraint, take c1.unsqueeze(dim=-1)
        # for >1 constraint, take torch.stack([c1, c2....], dim=-1)
        slack = torch.stack([c1, c2], dim=-1)
        
        return output, slack
    
initial_x_array = loadtxt("initial_x_10trials8dim_01bounds.csv", delimiter=',')
initial_x_array = initial_x_array.reshape(10, 18, 8)'''

'dimensions = 8\n    \nfrom botorch.test_functions.multi_objective import MW7\n\nMW7base = MW7(dim=dimensions, negate=True).to(**tkwargs)\n\nclass Problem_MW7(torch.nn.Module):\n    n_var = dimensions\n    n_obj = 2\n    n_constr = 2\n    \n    ref_point = torch.tensor([1.2,1.2], **tkwargs)\n    \n    bounds = MW7base.bounds    \n    \n    def evaluate(X):       \n        output = MW7base(X)\n        slack = -MW7base.evaluate_slack(X)\n       \n        return output, slack\n\nfrom botorch.test_functions.multi_objective import WeldedBeam\n\nWeldedBeambase = WeldedBeam(negate=True).to(**tkwargs)\n\nclass Problem_WeldedBeam(torch.nn.Module):\n    n_var = WeldedBeambase.dim\n    n_obj = WeldedBeambase.num_objectives\n    n_constr = WeldedBeambase.num_constraints\n    \n    ref_point = torch.tensor([40,0.015], **tkwargs)\n    \n    bounds = WeldedBeambase.bounds    \n    \n    def evaluate(X):       \n        output = WeldedBeambase(X)\n        slack = -WeldedBeambase.evaluate_slack(X)\n   

In [5]:
import joblib

initial_x_dict = joblib.load('initial_x_dict')

In [7]:
problemname = 'MW5'

N_TRIALS = 5
verbose = True
noise = 0.00

N_BATCH = 24
BATCH_SIZE = 8

for dimensions in range(8, 12+ 1):
    
    class MWBase(torch.nn.Module):   
        def g1(X, n_obj):
            d = dimensions
            n = d - n_obj

            z = torch.pow(X[..., n_obj - 1:], n)
            i = torch.arange(n_obj - 1, d, **tkwargs)

            exp = 1 - torch.exp(-10.0 * (z - 0.5 - i / (2 * d)) * (z - 0.5 - i / (2 * d)))
            distance = 1 + exp.sum(axis=1)
            return distance

        def g3(X, n_obj):
            contrib = 2.0 * torch.pow(
                X[..., n_obj - 1:] + (X[..., n_obj - 2:-1] - 0.5) * (X[..., n_obj - 2:-1] - 0.5) - 1.0, 2.0)

            distance = 1 + contrib.sum(axis=1)
            return distance

        def LA1(A, B, C, D, theta):
            return A * torch.pow(torch.sin(B * np.pi * torch.pow(theta, C)), D)

        def LA2(A, B, C, D, theta):
            return A * torch.pow(torch.sin(B * torch.pow(theta, C)), D)

    class problem(torch.nn.Module):
        n_var = dimensions
        n_obj = 2
        n_constr = 3

        ref_point = torch.tensor([1.2,1.2], **tkwargs)

        bounds = torch.vstack([torch.zeros(dimensions, **tkwargs),torch.ones(dimensions, **tkwargs)])

        def evaluate(X):
            # to maximise a postive output, stay positive
            # to minimise a positive output, force negative
            # to maximise a negative output (away from zero), force positive
            # to minimise a negative output (towards zero), stay negative

            g1 = MWBase.g3(X, n_obj=2)
            f1 = g1 * X[:, 0]
            f2 = g1 * torch.sqrt(1.0 - torch.pow(f1 / g1, 2.0))

            # constraint equations should take the form of c(n)<=0
            # should return <=0 values when constraints are active,
            # and >0 values when constraints are violated
            # take care if constraints are too restrictive, will cause early flatlining if all samples are rejected
            atan = torch.arctan(f2 / f1)
            c1 =  (f1 ** 2) + (f2 ** 2) - torch.pow(1.7 - MWBase.LA2(0.2, 2.0, 1.0, 1.0, atan), 2.0) 

            t = 0.5 * np.pi - 2 * torch.abs(atan - 0.25 * np.pi)
            c2 = torch.pow(1 + MWBase.LA2(0.5, 6.0, 3.0, 1.0, t), 2.0) - (f1 ** 2) - (f2 ** 2)
            c3 = torch.pow(1 - MWBase.LA2(0.45, 6.0, 3.0, 1.0, t), 2.0) - (f1 ** 2) - (f2 ** 2)

            output = torch.stack([-f1, -f2], dim=-1)
            slack = torch.stack([c1, c2, c3], dim=-1)

            return output, slack
        
    hv_list = []
    train_list = []

    # main loop for each trial/run, random_state will be trial number
    for trial in range(0, N_TRIALS):
        print(f"\nTrial {trial+1:>2} of {N_TRIALS} for problem {problemname} with d = {dimensions}\n", end="")

        # initialize with a 2*(d+1) sample set
        initial_x = torch.tensor(initial_x_dict[dimensions][trial], **tkwargs)

        hv, train = optimize_qnehvi(problem, problem.ref_point, initial_x,
                         N_BATCH=N_BATCH, BATCH_SIZE=BATCH_SIZE,
                         random_state=trial, noise=noise, verbose=verbose)
        hv_list.append(hv)
        train_list.append(train)

    savetxt(f"{problemname}_{dimensions}dim_hvs_qnehvi_{N_BATCH}by{BATCH_SIZE}_{N_TRIALS}trials.csv", hv_list, delimiter=',')    
    savetxt(f"{problemname}_{dimensions}dim_train_qnehvi_{N_BATCH}by{BATCH_SIZE}_{N_TRIALS}trials.csv", np.array(train_list).reshape(-1), delimiter=',')
        
print("All done!")


Trial  1 of 5 for problem MW5 with d = 8
Optimizing with Pure qNEHVI


C:\Users\lowki\anaconda3\lib\site-packages\gpytorch\lazy\lazy_tensor.py:1741: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\BatchLinearAlgebra.cpp:2189.)
  Linv = torch.triangular_solve(Eye, L, upper=False).solution


Batch  1 of 24: Hypervolume = 0.00, time = 6.30s.
Batch  2 of 24: Hypervolume = 0.00, time = 28.76s.
Batch  3 of 24: Hypervolume = 0.00, time = 49.15s.
Batch  4 of 24: Hypervolume = 0.14, time = 41.50s.
Batch  5 of 24: Hypervolume = 0.20, time = 44.65s.
Batch  6 of 24: Hypervolume = 0.20, time = 20.49s.
Batch  7 of 24: Hypervolume = 0.20, time = 23.00s.
Batch  8 of 24: Hypervolume = 0.20, time = 36.46s.
Batch  9 of 24: Hypervolume = 0.20, time = 12.83s.
Batch 10 of 24: Hypervolume = 0.20, time = 9.41s.
Batch 11 of 24: Hypervolume = 0.20, time = 35.71s.
Batch 12 of 24: Hypervolume = 0.20, time = 6.35s.
Batch 13 of 24: Hypervolume = 0.20, time = 39.08s.
Batch 14 of 24: Hypervolume = 0.20, time = 30.46s.
Batch 15 of 24: Hypervolume = 0.20, time = 17.23s.
Batch 16 of 24: Hypervolume = 0.24, time = 37.33s.
Batch 17 of 24: Hypervolume = 0.24, time = 6.10s.
Batch 18 of 24: Hypervolume = 0.24, time = 27.50s.
Batch 19 of 24: Hypervolume = 0.24, time = 31.35s.
Batch 20 of 24: Hypervolume = 0.24,

Batch  7 of 24: Hypervolume = 0.00, time = 12.15s.
Batch  8 of 24: Hypervolume = 0.00, time = 6.36s.
Batch  9 of 24: Hypervolume = 0.07, time = 7.22s.
Batch 10 of 24: Hypervolume = 0.07, time = 16.06s.
Batch 11 of 24: Hypervolume = 0.07, time = 11.34s.
Batch 12 of 24: Hypervolume = 0.07, time = 32.62s.
Batch 13 of 24: Hypervolume = 0.09, time = 6.32s.
Batch 14 of 24: Hypervolume = 0.09, time = 14.51s.
Batch 15 of 24: Hypervolume = 0.09, time = 5.50s.
Batch 16 of 24: Hypervolume = 0.12, time = 10.26s.
Batch 17 of 24: Hypervolume = 0.12, time = 5.15s.
Batch 18 of 24: Hypervolume = 0.12, time = 28.70s.
Batch 19 of 24: Hypervolume = 0.12, time = 11.68s.
Batch 20 of 24: Hypervolume = 0.12, time = 3.98s.
Batch 21 of 24: Hypervolume = 0.12, time = 17.13s.
Batch 22 of 24: Hypervolume = 0.12, time = 16.88s.
Batch 23 of 24: Hypervolume = 0.12, time = 12.31s.
Batch 24 of 24: Hypervolume = 0.12, time = 10.22s.
Time taken in total: 421.81s.

Trial  3 of 5 for problem MW5 with d = 9
Optimizing with 

Batch 13 of 24: Hypervolume = 0.05, time = 25.72s.
Batch 14 of 24: Hypervolume = 0.05, time = 18.11s.
Batch 15 of 24: Hypervolume = 0.05, time = 6.57s.
Batch 16 of 24: Hypervolume = 0.05, time = 4.71s.
Batch 17 of 24: Hypervolume = 0.05, time = 30.23s.
Batch 18 of 24: Hypervolume = 0.05, time = 26.50s.
Batch 19 of 24: Hypervolume = 0.05, time = 32.79s.
Batch 20 of 24: Hypervolume = 0.05, time = 35.08s.
Batch 21 of 24: Hypervolume = 0.05, time = 7.79s.
Batch 22 of 24: Hypervolume = 0.05, time = 29.62s.
Batch 23 of 24: Hypervolume = 0.07, time = 5.40s.
Batch 24 of 24: Hypervolume = 0.07, time = 36.94s.
Time taken in total: 597.20s.

Trial  4 of 5 for problem MW5 with d = 10
Optimizing with Pure qNEHVI
Batch  1 of 24: Hypervolume = 0.00, time = 19.13s.
Batch  2 of 24: Hypervolume = 0.00, time = 31.86s.
Batch  3 of 24: Hypervolume = 0.00, time = 32.79s.
Batch  4 of 24: Hypervolume = 0.00, time = 5.93s.
Batch  5 of 24: Hypervolume = 0.00, time = 4.46s.
Batch  6 of 24: Hypervolume = 0.00, ti

Batch 19 of 24: Hypervolume = 0.00, time = 12.06s.
Batch 20 of 24: Hypervolume = 0.00, time = 5.45s.
Batch 21 of 24: Hypervolume = 0.00, time = 37.88s.
Batch 22 of 24: Hypervolume = 0.00, time = 5.26s.
Batch 23 of 24: Hypervolume = 0.00, time = 3.73s.
Batch 24 of 24: Hypervolume = 0.00, time = 3.56s.
Time taken in total: 398.49s.

Trial  5 of 5 for problem MW5 with d = 11
Optimizing with Pure qNEHVI
Batch  1 of 24: Hypervolume = 0.00, time = 26.21s.
Batch  2 of 24: Hypervolume = 0.00, time = 8.54s.
Batch  3 of 24: Hypervolume = 0.00, time = 45.58s.
Batch  4 of 24: Hypervolume = 0.00, time = 8.14s.
Batch  5 of 24: Hypervolume = 0.00, time = 12.11s.
Batch  6 of 24: Hypervolume = 0.00, time = 3.95s.
Batch  7 of 24: Hypervolume = 0.00, time = 16.98s.
Batch  8 of 24: Hypervolume = 0.00, time = 8.28s.
Batch  9 of 24: Hypervolume = 0.00, time = 33.30s.
Batch 10 of 24: Hypervolume = 0.00, time = 28.65s.
Batch 11 of 24: Hypervolume = 0.00, time = 3.80s.
Batch 12 of 24: Hypervolume = 0.00, time 

In [6]:
problemname = 'MW5'

N_TRIALS = 5
verbose = True
noise = 0.00

N_BATCH = 24
BATCH_SIZE = 8

for dimensions in range(10, 12+ 1):

    class MWBase(torch.nn.Module):   
        def g1(X, n_obj):
            d = dimensions
            n = d - n_obj

            z = torch.pow(X[..., n_obj - 1:], n)
            i = torch.arange(n_obj - 1, d, **tkwargs)

            exp = 1 - torch.exp(-10.0 * (z - 0.5 - i / (2 * d)) * (z - 0.5 - i / (2 * d)))
            distance = 1 + exp.sum(axis=1)
            return distance

        def g3(X, n_obj):
            contrib = 2.0 * torch.pow(
                X[..., n_obj - 1:] + (X[..., n_obj - 2:-1] - 0.5) * (X[..., n_obj - 2:-1] - 0.5) - 1.0, 2.0)

            distance = 1 + contrib.sum(axis=1)
            return distance

        def LA1(A, B, C, D, theta):
            return A * torch.pow(torch.sin(B * np.pi * torch.pow(theta, C)), D)

        def LA2(A, B, C, D, theta):
            return A * torch.pow(torch.sin(B * torch.pow(theta, C)), D)

    class problem(torch.nn.Module):
        n_var = dimensions
        n_obj = 2
        n_constr = 3

        ref_point = torch.tensor([1.2,1.2], **tkwargs)

        bounds = torch.vstack([torch.zeros(dimensions, **tkwargs),torch.ones(dimensions, **tkwargs)])

        def evaluate(X):
            # to maximise a postive output, stay positive
            # to minimise a positive output, force negative
            # to maximise a negative output (away from zero), force positive
            # to minimise a negative output (towards zero), stay negative

            g1 = MWBase.g3(X, n_obj=2)
            f1 = g1 * X[:, 0]
            f2 = g1 * torch.sqrt(1.0 - torch.pow(f1 / g1, 2.0))

            # constraint equations should take the form of c(n)<=0
            # should return <=0 values when constraints are active,
            # and >0 values when constraints are violated
            # take care if constraints are too restrictive, will cause early flatlining if all samples are rejected
            atan = torch.arctan(f2 / f1)
            c1 =  (f1 ** 2) + (f2 ** 2) - torch.pow(1.7 - MWBase.LA2(0.2, 2.0, 1.0, 1.0, atan), 2.0) 

            t = 0.5 * np.pi - 2 * torch.abs(atan - 0.25 * np.pi)
            c2 = torch.pow(1 + MWBase.LA2(0.5, 6.0, 3.0, 1.0, t), 2.0) - (f1 ** 2) - (f2 ** 2)
            c3 = torch.pow(1 - MWBase.LA2(0.45, 6.0, 3.0, 1.0, t), 2.0) - (f1 ** 2) - (f2 ** 2)

            output = torch.stack([-f1, -f2], dim=-1)
            slack = torch.stack([c1, c2, c3], dim=-1)

            return output, slack
        
    hv_list = []
    train_list = []

    # main loop for each trial/run, random_state will be trial number
    for trial in range(0, N_TRIALS):
        print(f"\nTrial {trial+1:>2} of {N_TRIALS} for problem {problemname} with d = {dimensions}\n", end="")

        # initialize with a 2*(d+1) sample set
        initial_x = torch.tensor(initial_x_dict[dimensions][trial], **tkwargs)

        hv, train = optimize_hybrid_nsga(problem, problem.ref_point, initial_x,
                         N_BATCH=N_BATCH, BATCH_SIZE=BATCH_SIZE,
                         random_state=trial, noise=noise, verbose=verbose)
        hv_list.append(hv)
        train_list.append(train)

    savetxt(f"{problemname}_{dimensions}dim_hvs_hybrid_nsga_{N_BATCH}by{BATCH_SIZE}_{N_TRIALS}trials.csv", hv_list, delimiter=',')    
    savetxt(f"{problemname}_{dimensions}dim_train_hybrid_nsga_{N_BATCH}by{BATCH_SIZE}_{N_TRIALS}trials.csv", np.array(train_list).reshape(-1), delimiter=',')
        
print("All done!")


Trial  1 of 5 for problem MW5 with d = 10
Optimizing with Hybrid qNEHVI + U-NSGA-III


C:\Users\lowki\anaconda3\lib\site-packages\gpytorch\lazy\lazy_tensor.py:1741: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\BatchLinearAlgebra.cpp:2189.)
  Linv = torch.triangular_solve(Eye, L, upper=False).solution


Batch  1 of 24: Hypervolume = 0.00, time = 14.28s.
Batch  2 of 24: Hypervolume = 0.00, time = 15.19s.
Batch  3 of 24: Hypervolume = 0.15, time = 29.20s.
Batch  4 of 24: Hypervolume = 0.15, time = 53.47s.
Batch  5 of 24: Hypervolume = 0.15, time = 15.92s.
Batch  6 of 24: Hypervolume = 0.15, time = 24.56s.
Batch  7 of 24: Hypervolume = 0.15, time = 13.88s.
Batch  8 of 24: Hypervolume = 0.24, time = 37.93s.
Batch  9 of 24: Hypervolume = 0.26, time = 30.60s.
Batch 10 of 24: Hypervolume = 0.26, time = 16.80s.
Batch 11 of 24: Hypervolume = 0.26, time = 20.47s.
Batch 12 of 24: Hypervolume = 0.26, time = 21.61s.
Batch 13 of 24: Hypervolume = 0.32, time = 14.15s.
Batch 14 of 24: Hypervolume = 0.32, time = 13.00s.
Batch 15 of 24: Hypervolume = 0.32, time = 27.78s.
Batch 16 of 24: Hypervolume = 0.32, time = 13.07s.
Batch 17 of 24: Hypervolume = 0.32, time = 12.56s.
Batch 18 of 24: Hypervolume = 0.33, time = 28.27s.
Batch 19 of 24: Hypervolume = 0.34, time = 13.22s.
Batch 20 of 24: Hypervolume = 0

Batch  5 of 24: Hypervolume = 0.18, time = 14.20s.
Batch  6 of 24: Hypervolume = 0.18, time = 11.68s.
Batch  7 of 24: Hypervolume = 0.18, time = 11.39s.
Batch  8 of 24: Hypervolume = 0.22, time = 20.71s.
Batch  9 of 24: Hypervolume = 0.26, time = 11.26s.
Batch 10 of 24: Hypervolume = 0.31, time = 17.60s.
Batch 11 of 24: Hypervolume = 0.31, time = 11.51s.
Batch 12 of 24: Hypervolume = 0.31, time = 11.56s.
Batch 13 of 24: Hypervolume = 0.31, time = 11.93s.
Batch 14 of 24: Hypervolume = 0.32, time = 11.39s.
Batch 15 of 24: Hypervolume = 0.32, time = 11.82s.
Batch 16 of 24: Hypervolume = 0.35, time = 11.90s.
Batch 17 of 24: Hypervolume = 0.37, time = 11.73s.
Batch 18 of 24: Hypervolume = 0.37, time = 11.31s.
Batch 19 of 24: Hypervolume = 0.38, time = 12.01s.
Batch 20 of 24: Hypervolume = 0.39, time = 12.29s.
Batch 21 of 24: Hypervolume = 0.39, time = 12.76s.
Batch 22 of 24: Hypervolume = 0.39, time = 12.42s.
Batch 23 of 24: Hypervolume = 0.39, time = 12.98s.
Batch 24 of 24: Hypervolume = 0

Batch  9 of 24: Hypervolume = 0.21, time = 10.46s.
Batch 10 of 24: Hypervolume = 0.22, time = 10.15s.
Batch 11 of 24: Hypervolume = 0.22, time = 9.94s.
Batch 12 of 24: Hypervolume = 0.23, time = 10.40s.
Batch 13 of 24: Hypervolume = 0.26, time = 18.27s.
Batch 14 of 24: Hypervolume = 0.26, time = 10.39s.
Batch 15 of 24: Hypervolume = 0.28, time = 10.20s.
Batch 16 of 24: Hypervolume = 0.28, time = 10.07s.
Batch 17 of 24: Hypervolume = 0.28, time = 10.38s.
Batch 18 of 24: Hypervolume = 0.28, time = 10.01s.
Batch 19 of 24: Hypervolume = 0.29, time = 10.13s.
Batch 20 of 24: Hypervolume = 0.29, time = 10.89s.
Batch 21 of 24: Hypervolume = 0.32, time = 10.53s.
Batch 22 of 24: Hypervolume = 0.32, time = 10.78s.
Batch 23 of 24: Hypervolume = 0.32, time = 10.45s.
Batch 24 of 24: Hypervolume = 0.32, time = 12.26s.
Time taken in total: 335.63s.

Trial  4 of 5 for problem MW5 with d = 12
Optimizing with Hybrid qNEHVI + U-NSGA-III
Batch  1 of 24: Hypervolume = 0.00, time = 8.04s.
Batch  2 of 24: Hyp

In [6]:
problemname = 'MW3'

N_TRIALS = 5
verbose = True
noise = 0.00

N_BATCH = 24
BATCH_SIZE = 8

for dimensions in range(7, 12+ 1):

    class MWBase(torch.nn.Module):   
        def g1(X, n_obj):
            d = dimensions
            n = d - n_obj

            z = torch.pow(X[..., n_obj - 1:], n)
            i = torch.arange(n_obj - 1, d, **tkwargs)

            exp = 1 - torch.exp(-10.0 * (z - 0.5 - i / (2 * d)) * (z - 0.5 - i / (2 * d)))
            distance = 1 + exp.sum(axis=1)
            return distance

        def g3(X, n_obj):
            contrib = 2.0 * torch.pow(
                X[..., n_obj - 1:] + (X[..., n_obj - 2:-1] - 0.5) * (X[..., n_obj - 2:-1] - 0.5) - 1.0, 2.0)

            distance = 1 + contrib.sum(axis=1)
            return distance

        def LA1(A, B, C, D, theta):
            return A * torch.pow(torch.sin(B * np.pi * torch.pow(theta, C)), D)

        def LA2(A, B, C, D, theta):
            return A * torch.pow(torch.sin(B * torch.pow(theta, C)), D)

    class problem(torch.nn.Module):
        # must define these!
        n_var = dimensions
        n_obj = 2
        n_constr = 2 # inequality constraints only!

        ref_point = torch.tensor([1.1,1.1], **tkwargs)

        # input bounds, don't forget to change according to n_var!  
        bounds = torch.vstack([torch.zeros(dimensions, **tkwargs),torch.ones(dimensions, **tkwargs)])

        def evaluate(X):

            # objectives are defined by f1, f2....
            # if you have complex equations, you can break it up and place them in ProblemBase for convenience, not compulsory
            # otherwise, you could place an oracle here that outputs the objective
            g = MWBase.g3(X, n_obj=2)
            f1 = X[:, 0]
            f2 = g * (1.0 - f1 / g)

            # inequality constraints should take the form of c(n)<=0
            # for equality constraints, either use the equality constr argument, or rewrite into inequality form (preferred)
            # if no constraints, write as c1 = 0 * X[..., 0], and set n_constr=1     
            c1 = f1 + f2 - 1.05 - MWBase.LA1(0.45, 0.75, 1.0, 6.0, np.sqrt(2.0) * f2 - np.sqrt(2.0) * f1)
            c2 = 0.85 - f1 - f2 + MWBase.LA1(0.3, 0.75, 1.0, 2.0, np.sqrt(2.0) * f2 - np.sqrt(2.0) * f1)

            # botorch assumes maximisation
            # to maximise a postive output, stay positive
            # to minimise a positive output, force negative
            # to maximise a negative output (away from zero), force positive
            # to minimise a negative output (towards zero), stay negative
            output = torch.stack([-f1, -f2], dim=-1)

            # for 1 constraint, take c1.unsqueeze(dim=-1)
            # for >1 constraint, take torch.stack([c1, c2....], dim=-1)
            slack = torch.stack([c1, c2], dim=-1)

            return output, slack
        
    hv_list = []
    train_list = []

    # main loop for each trial/run, random_state will be trial number
    for trial in range(0, N_TRIALS):
        print(f"\nTrial {trial+1:>2} of {N_TRIALS} for problem {problemname} with d = {dimensions}\n", end="")

        # initialize with a 2*(d+1) sample set
        initial_x = torch.tensor(initial_x_dict[dimensions][trial], **tkwargs)

        hv, train = optimize_hybrid_nsga(problem, problem.ref_point, initial_x,
                         N_BATCH=N_BATCH, BATCH_SIZE=BATCH_SIZE,
                         random_state=trial, noise=noise, verbose=verbose)
        hv_list.append(hv)
        train_list.append(train)

    savetxt(f"{problemname}_{dimensions}dim_hvs_hybrid_nsga_{N_BATCH}by{BATCH_SIZE}_{N_TRIALS}trials.csv", hv_list, delimiter=',')    
    savetxt(f"{problemname}_{dimensions}dim_train_hybrid_nsga_{N_BATCH}by{BATCH_SIZE}_{N_TRIALS}trials.csv", np.array(train_list).reshape(-1), delimiter=',')
        
print("All done!")


Trial  1 of 5 for problem MW3 with d = 7
Optimizing with Hybrid qNEHVI + U-NSGA-III


C:\Users\lowki\anaconda3\lib\site-packages\gpytorch\lazy\lazy_tensor.py:1741: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\BatchLinearAlgebra.cpp:2189.)
  Linv = torch.triangular_solve(Eye, L, upper=False).solution


Batch  1 of 24: Hypervolume = 0.00, time = 12.96s.
Batch  2 of 24: Hypervolume = 0.00, time = 15.62s.
Batch  3 of 24: Hypervolume = 0.14, time = 14.40s.
Batch  4 of 24: Hypervolume = 0.19, time = 33.53s.
Batch  5 of 24: Hypervolume = 0.35, time = 22.66s.
Batch  6 of 24: Hypervolume = 0.38, time = 12.45s.
Batch  7 of 24: Hypervolume = 0.49, time = 29.86s.
Batch  8 of 24: Hypervolume = 0.56, time = 17.46s.
Batch  9 of 24: Hypervolume = 0.57, time = 12.83s.
Batch 10 of 24: Hypervolume = 0.57, time = 18.53s.
Batch 11 of 24: Hypervolume = 0.59, time = 19.86s.
Batch 12 of 24: Hypervolume = 0.61, time = 15.97s.
Batch 13 of 24: Hypervolume = 0.61, time = 15.65s.
Batch 14 of 24: Hypervolume = 0.62, time = 11.04s.
Batch 15 of 24: Hypervolume = 0.62, time = 15.48s.
Batch 16 of 24: Hypervolume = 0.62, time = 15.59s.
Batch 17 of 24: Hypervolume = 0.63, time = 30.69s.
Batch 18 of 24: Hypervolume = 0.63, time = 14.37s.
Batch 19 of 24: Hypervolume = 0.63, time = 18.26s.
Batch 20 of 24: Hypervolume = 0

Batch  5 of 24: Hypervolume = 0.36, time = 12.20s.
Batch  6 of 24: Hypervolume = 0.49, time = 12.39s.
Batch  7 of 24: Hypervolume = 0.52, time = 24.44s.
Batch  8 of 24: Hypervolume = 0.55, time = 11.58s.
Batch  9 of 24: Hypervolume = 0.57, time = 23.10s.
Batch 10 of 24: Hypervolume = 0.59, time = 13.40s.
Batch 11 of 24: Hypervolume = 0.59, time = 13.01s.
Batch 12 of 24: Hypervolume = 0.60, time = 16.67s.
Batch 13 of 24: Hypervolume = 0.61, time = 10.44s.
Batch 14 of 24: Hypervolume = 0.61, time = 15.50s.
Batch 15 of 24: Hypervolume = 0.62, time = 25.59s.
Batch 16 of 24: Hypervolume = 0.62, time = 10.67s.
Batch 17 of 24: Hypervolume = 0.62, time = 26.67s.
Batch 18 of 24: Hypervolume = 0.63, time = 12.11s.
Batch 19 of 24: Hypervolume = 0.63, time = 16.48s.
Batch 20 of 24: Hypervolume = 0.63, time = 20.63s.
Batch 21 of 24: Hypervolume = 0.63, time = 15.09s.
Batch 22 of 24: Hypervolume = 0.63, time = 12.12s.
Batch 23 of 24: Hypervolume = 0.63, time = 16.61s.
Batch 24 of 24: Hypervolume = 0

Batch  9 of 24: Hypervolume = 0.57, time = 14.95s.
Batch 10 of 24: Hypervolume = 0.58, time = 15.73s.
Batch 11 of 24: Hypervolume = 0.58, time = 8.53s.
Batch 12 of 24: Hypervolume = 0.58, time = 12.53s.
Batch 13 of 24: Hypervolume = 0.59, time = 9.26s.
Batch 14 of 24: Hypervolume = 0.60, time = 9.67s.
Batch 15 of 24: Hypervolume = 0.60, time = 22.66s.
Batch 16 of 24: Hypervolume = 0.61, time = 9.44s.
Batch 17 of 24: Hypervolume = 0.61, time = 31.40s.
Batch 18 of 24: Hypervolume = 0.62, time = 15.82s.
Batch 19 of 24: Hypervolume = 0.62, time = 10.36s.
Batch 20 of 24: Hypervolume = 0.62, time = 9.73s.
Batch 21 of 24: Hypervolume = 0.62, time = 9.53s.
Batch 22 of 24: Hypervolume = 0.62, time = 9.67s.
Batch 23 of 24: Hypervolume = 0.63, time = 21.04s.
Batch 24 of 24: Hypervolume = 0.63, time = 10.61s.
Time taken in total: 383.69s.

Trial  4 of 5 for problem MW3 with d = 9
Optimizing with Hybrid qNEHVI + U-NSGA-III
Batch  1 of 24: Hypervolume = 0.00, time = 19.10s.
Batch  2 of 24: Hypervolu

Batch 13 of 24: Hypervolume = 0.55, time = 10.11s.
Batch 14 of 24: Hypervolume = 0.56, time = 19.93s.
Batch 15 of 24: Hypervolume = 0.57, time = 34.74s.
Batch 16 of 24: Hypervolume = 0.58, time = 11.56s.
Batch 17 of 24: Hypervolume = 0.58, time = 9.14s.
Batch 18 of 24: Hypervolume = 0.59, time = 9.35s.
Batch 19 of 24: Hypervolume = 0.59, time = 9.25s.
Batch 20 of 24: Hypervolume = 0.60, time = 9.25s.
Batch 21 of 24: Hypervolume = 0.60, time = 10.72s.
Batch 22 of 24: Hypervolume = 0.60, time = 19.56s.
Batch 23 of 24: Hypervolume = 0.61, time = 13.29s.
Batch 24 of 24: Hypervolume = 0.61, time = 10.61s.
Time taken in total: 356.07s.

Trial  5 of 5 for problem MW3 with d = 10
Optimizing with Hybrid qNEHVI + U-NSGA-III
Batch  1 of 24: Hypervolume = 0.00, time = 8.37s.
Batch  2 of 24: Hypervolume = 0.00, time = 9.23s.
Batch  3 of 24: Hypervolume = 0.00, time = 9.92s.
Batch  4 of 24: Hypervolume = 0.20, time = 17.98s.
Batch  5 of 24: Hypervolume = 0.30, time = 14.92s.
Batch  6 of 24: Hypervol

Batch 18 of 24: Hypervolume = 0.46, time = 11.12s.
Batch 19 of 24: Hypervolume = 0.46, time = 10.48s.
Batch 20 of 24: Hypervolume = 0.49, time = 10.76s.
Batch 21 of 24: Hypervolume = 0.51, time = 10.37s.
Batch 22 of 24: Hypervolume = 0.51, time = 10.31s.
Batch 23 of 24: Hypervolume = 0.52, time = 11.43s.
Batch 24 of 24: Hypervolume = 0.54, time = 10.79s.
Time taken in total: 274.15s.

Trial  1 of 5 for problem MW3 with d = 12
Optimizing with Hybrid qNEHVI + U-NSGA-III
Batch  1 of 24: Hypervolume = 0.00, time = 9.09s.
Batch  2 of 24: Hypervolume = 0.00, time = 9.76s.
Batch  3 of 24: Hypervolume = 0.00, time = 10.69s.
Batch  4 of 24: Hypervolume = 0.00, time = 20.64s.
Batch  5 of 24: Hypervolume = 0.21, time = 10.76s.
Batch  6 of 24: Hypervolume = 0.25, time = 21.28s.
Batch  7 of 24: Hypervolume = 0.25, time = 11.14s.
Batch  8 of 24: Hypervolume = 0.27, time = 11.04s.
Batch  9 of 24: Hypervolume = 0.33, time = 12.29s.
Batch 10 of 24: Hypervolume = 0.34, time = 16.56s.
Batch 11 of 24: Hyp

In [6]:
from botorch.test_functions.multi_objective import MW7

problemname = 'MW7'

N_TRIALS = 5
verbose = True
noise = 0.00

N_BATCH = 24
BATCH_SIZE = 8

for dimensions in range(2, 6+ 1):

    MW7base = MW7(dim=dimensions, negate=True).to(**tkwargs)

    class problem(torch.nn.Module):
        n_var = dimensions
        n_obj = 2
        n_constr = 2

        ref_point = torch.tensor([1.2,1.2], **tkwargs)

        bounds = MW7base.bounds    

        def evaluate(X):       
            output = MW7base(X)
            slack = -MW7base.evaluate_slack(X)

            return output, slack
        
    hv_list = []
    train_list = []

    # main loop for each trial/run, random_state will be trial number
    for trial in range(0, N_TRIALS):
        print(f"\nTrial {trial+1:>2} of {N_TRIALS} for problem {problemname} with d = {dimensions}\n", end="")

        # initialize with a 2*(d+1) sample set
        initial_x = torch.tensor(initial_x_dict[dimensions][trial], **tkwargs)

        hv, train = optimize_hybrid_moead(problem, problem.ref_point, initial_x,
                         N_BATCH=N_BATCH, BATCH_SIZE=BATCH_SIZE,
                         random_state=trial, noise=noise, verbose=verbose)
        hv_list.append(hv)
        train_list.append(train)

    savetxt(f"{problemname}_{dimensions}dim_hvs_hybrid_moead_{N_BATCH}by{BATCH_SIZE}_{N_TRIALS}trials.csv", hv_list, delimiter=',')    
    savetxt(f"{problemname}_{dimensions}dim_train_hybrid_moead_{N_BATCH}by{BATCH_SIZE}_{N_TRIALS}trials.csv", np.array(train_list).reshape(-1), delimiter=',')
        
print("All done!")


Trial  1 of 5 for problem MW7 with d = 2
Optimizing with Hybrid qNEHVI + MOEAD


C:\Users\user\anaconda3\lib\site-packages\gpytorch\lazy\lazy_tensor.py:1741: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\BatchLinearAlgebra.cpp:2189.)
  Linv = torch.triangular_solve(Eye, L, upper=False).solution


Batch  1 of 24: Hypervolume = 0.15, time = 27.79s.
Batch  2 of 24: Hypervolume = 0.15, time = 24.85s.
Batch  3 of 24: Hypervolume = 0.24, time = 11.95s.
Batch  4 of 24: Hypervolume = 0.24, time = 14.35s.
Batch  5 of 24: Hypervolume = 0.36, time = 13.70s.
Batch  6 of 24: Hypervolume = 0.41, time = 8.18s.
Batch  7 of 24: Hypervolume = 0.43, time = 9.74s.
Batch  8 of 24: Hypervolume = 0.45, time = 9.29s.
Batch  9 of 24: Hypervolume = 0.46, time = 13.08s.
Batch 10 of 24: Hypervolume = 0.47, time = 10.18s.
Batch 11 of 24: Hypervolume = 0.48, time = 8.25s.
Batch 12 of 24: Hypervolume = 0.48, time = 9.54s.
Batch 13 of 24: Hypervolume = 0.48, time = 13.29s.
Batch 14 of 24: Hypervolume = 0.49, time = 8.36s.
Batch 15 of 24: Hypervolume = 0.49, time = 8.42s.
Batch 16 of 24: Hypervolume = 0.49, time = 11.64s.
Batch 17 of 24: Hypervolume = 0.49, time = 10.41s.
Batch 18 of 24: Hypervolume = 0.49, time = 10.62s.
Batch 19 of 24: Hypervolume = 0.49, time = 11.29s.
Batch 20 of 24: Hypervolume = 0.49, ti

Batch  6 of 24: Hypervolume = 0.40, time = 8.50s.
Batch  7 of 24: Hypervolume = 0.40, time = 8.54s.
Batch  8 of 24: Hypervolume = 0.42, time = 33.14s.
Batch  9 of 24: Hypervolume = 0.44, time = 23.80s.
Batch 10 of 24: Hypervolume = 0.45, time = 11.98s.
Batch 11 of 24: Hypervolume = 0.45, time = 28.16s.
Batch 12 of 24: Hypervolume = 0.46, time = 11.32s.
Batch 13 of 24: Hypervolume = 0.47, time = 12.34s.
Batch 14 of 24: Hypervolume = 0.47, time = 8.21s.
Batch 15 of 24: Hypervolume = 0.48, time = 17.37s.
Batch 16 of 24: Hypervolume = 0.48, time = 8.58s.
Batch 17 of 24: Hypervolume = 0.48, time = 9.45s.
Batch 18 of 24: Hypervolume = 0.48, time = 8.54s.
Batch 19 of 24: Hypervolume = 0.49, time = 10.96s.
Batch 20 of 24: Hypervolume = 0.49, time = 11.47s.
Batch 21 of 24: Hypervolume = 0.49, time = 10.56s.
Batch 22 of 24: Hypervolume = 0.49, time = 10.93s.
Batch 23 of 24: Hypervolume = 0.49, time = 14.94s.
Batch 24 of 24: Hypervolume = 0.49, time = 10.47s.
Time taken in total: 376.57s.

Trial 

Batch 11 of 24: Hypervolume = 0.43, time = 32.93s.
Batch 12 of 24: Hypervolume = 0.45, time = 25.96s.
Batch 13 of 24: Hypervolume = 0.45, time = 33.26s.
Batch 14 of 24: Hypervolume = 0.46, time = 9.41s.
Batch 15 of 24: Hypervolume = 0.46, time = 30.77s.
Batch 16 of 24: Hypervolume = 0.46, time = 9.38s.
Batch 17 of 24: Hypervolume = 0.46, time = 10.43s.
Batch 18 of 24: Hypervolume = 0.47, time = 10.59s.
Batch 19 of 24: Hypervolume = 0.47, time = 10.00s.
Batch 20 of 24: Hypervolume = 0.47, time = 16.85s.
Batch 21 of 24: Hypervolume = 0.47, time = 33.46s.
Batch 22 of 24: Hypervolume = 0.47, time = 14.26s.
Batch 23 of 24: Hypervolume = 0.47, time = 25.49s.
Batch 24 of 24: Hypervolume = 0.47, time = 12.15s.
Time taken in total: 467.31s.

Trial  4 of 5 for problem MW7 with d = 4
Optimizing with Hybrid qNEHVI + MOEAD
Batch  1 of 24: Hypervolume = 0.29, time = 15.20s.
Batch  2 of 24: Hypervolume = 0.30, time = 30.23s.
Batch  3 of 24: Hypervolume = 0.30, time = 27.98s.
Batch  4 of 24: Hypervolu

Batch 15 of 24: Hypervolume = 0.44, time = 30.95s.
Batch 16 of 24: Hypervolume = 0.45, time = 22.33s.
Batch 17 of 24: Hypervolume = 0.45, time = 14.62s.
Batch 18 of 24: Hypervolume = 0.45, time = 20.60s.
Batch 19 of 24: Hypervolume = 0.45, time = 41.60s.
Batch 20 of 24: Hypervolume = 0.46, time = 28.03s.
Batch 21 of 24: Hypervolume = 0.46, time = 28.90s.
Batch 22 of 24: Hypervolume = 0.46, time = 12.65s.
Batch 23 of 24: Hypervolume = 0.46, time = 13.59s.
Batch 24 of 24: Hypervolume = 0.46, time = 27.14s.
Time taken in total: 531.02s.

Trial  5 of 5 for problem MW7 with d = 5
Optimizing with Hybrid qNEHVI + MOEAD
Batch  1 of 24: Hypervolume = 0.06, time = 38.61s.
Batch  2 of 24: Hypervolume = 0.06, time = 31.57s.
Batch  3 of 24: Hypervolume = 0.06, time = 11.19s.
Batch  4 of 24: Hypervolume = 0.23, time = 21.37s.
Batch  5 of 24: Hypervolume = 0.25, time = 27.24s.
Batch  6 of 24: Hypervolume = 0.26, time = 9.44s.
Batch  7 of 24: Hypervolume = 0.26, time = 8.63s.
Batch  8 of 24: Hypervolu

Batch 19 of 24: Hypervolume = 0.45, time = 27.39s.
Batch 20 of 24: Hypervolume = 0.45, time = 33.40s.
Batch 21 of 24: Hypervolume = 0.46, time = 9.70s.
Batch 22 of 24: Hypervolume = 0.46, time = 28.19s.
Batch 23 of 24: Hypervolume = 0.46, time = 17.29s.
Batch 24 of 24: Hypervolume = 0.46, time = 18.03s.
Time taken in total: 570.13s.
All done!


In [6]:
from botorch.test_functions.multi_objective import MW7

problemname = 'MW7'

N_TRIALS = 5
verbose = True
noise = 0.00

N_BATCH = 24
BATCH_SIZE = 8

for dimensions in range(8, 9+ 1):

    MW7base = MW7(dim=dimensions, negate=True).to(**tkwargs)

    class problem(torch.nn.Module):
        n_var = dimensions
        n_obj = 2
        n_constr = 2

        ref_point = torch.tensor([1.2,1.2], **tkwargs)

        bounds = MW7base.bounds    

        def evaluate(X):       
            output = MW7base(X)
            slack = -MW7base.evaluate_slack(X)

            return output, slack
        
    hv_list = []
    train_list = []

    # main loop for each trial/run, random_state will be trial number
    for trial in range(0, N_TRIALS):
        print(f"\nTrial {trial+1:>2} of {N_TRIALS} for problem {problemname} with d = {dimensions}\n", end="")

        # initialize with a 2*(d+1) sample set
        initial_x = torch.tensor(initial_x_dict[dimensions][trial], **tkwargs)

        hv, train = optimize_hybrid_moead(problem, problem.ref_point, initial_x,
                         N_BATCH=N_BATCH, BATCH_SIZE=BATCH_SIZE,
                         random_state=trial, noise=noise, verbose=verbose)
        hv_list.append(hv)
        train_list.append(train)

    savetxt(f"{problemname}_{dimensions}dim_hvs_hybrid_moead_{N_BATCH}by{BATCH_SIZE}_{N_TRIALS}trials.csv", hv_list, delimiter=',')    
    savetxt(f"{problemname}_{dimensions}dim_train_hybrid_moead_{N_BATCH}by{BATCH_SIZE}_{N_TRIALS}trials.csv", np.array(train_list).reshape(-1), delimiter=',')
        
print("All done!")


Trial  1 of 5 for problem MW7 with d = 8
Optimizing with Hybrid qNEHVI + MOEAD


C:\Users\user\anaconda3\lib\site-packages\gpytorch\lazy\lazy_tensor.py:1741: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\BatchLinearAlgebra.cpp:2189.)
  Linv = torch.triangular_solve(Eye, L, upper=False).solution


Batch  1 of 24: Hypervolume = 0.00, time = 22.06s.
Batch  2 of 24: Hypervolume = 0.00, time = 6.33s.
Batch  3 of 24: Hypervolume = 0.00, time = 14.23s.
Batch  4 of 24: Hypervolume = 0.05, time = 13.52s.
Batch  5 of 24: Hypervolume = 0.05, time = 7.93s.
Batch  6 of 24: Hypervolume = 0.06, time = 17.53s.
Batch  7 of 24: Hypervolume = 0.15, time = 25.29s.
Batch  8 of 24: Hypervolume = 0.16, time = 23.22s.
Batch  9 of 24: Hypervolume = 0.16, time = 14.24s.
Batch 10 of 24: Hypervolume = 0.24, time = 28.41s.
Batch 11 of 24: Hypervolume = 0.30, time = 10.78s.
Batch 12 of 24: Hypervolume = 0.33, time = 26.95s.
Batch 13 of 24: Hypervolume = 0.34, time = 12.88s.
Batch 14 of 24: Hypervolume = 0.37, time = 24.30s.
Batch 15 of 24: Hypervolume = 0.38, time = 11.79s.
Batch 16 of 24: Hypervolume = 0.39, time = 11.55s.
Batch 17 of 24: Hypervolume = 0.42, time = 7.39s.
Batch 18 of 24: Hypervolume = 0.42, time = 9.35s.
Batch 19 of 24: Hypervolume = 0.43, time = 17.86s.
Batch 20 of 24: Hypervolume = 0.43,

Batch  6 of 24: Hypervolume = 0.18, time = 19.97s.
Batch  7 of 24: Hypervolume = 0.18, time = 30.23s.
Batch  8 of 24: Hypervolume = 0.20, time = 23.92s.
Batch  9 of 24: Hypervolume = 0.23, time = 13.33s.
Batch 10 of 24: Hypervolume = 0.28, time = 22.58s.
Batch 11 of 24: Hypervolume = 0.31, time = 13.74s.
Batch 12 of 24: Hypervolume = 0.32, time = 8.07s.
Batch 13 of 24: Hypervolume = 0.33, time = 15.58s.
Batch 14 of 24: Hypervolume = 0.37, time = 8.42s.
Batch 15 of 24: Hypervolume = 0.37, time = 16.70s.
Batch 16 of 24: Hypervolume = 0.39, time = 20.02s.
Batch 17 of 24: Hypervolume = 0.40, time = 7.94s.
Batch 18 of 24: Hypervolume = 0.41, time = 7.59s.
Batch 19 of 24: Hypervolume = 0.41, time = 16.45s.
Batch 20 of 24: Hypervolume = 0.41, time = 11.37s.
Batch 21 of 24: Hypervolume = 0.41, time = 7.78s.
Batch 22 of 24: Hypervolume = 0.41, time = 7.91s.
Batch 23 of 24: Hypervolume = 0.42, time = 12.22s.
Batch 24 of 24: Hypervolume = 0.42, time = 13.80s.
Time taken in total: 347.05s.

Trial 